# **Image Restoration**

## Setup Environment

In [1]:
%load_ext autoreload
%autoreload 2
import os

# Test if it's running in Google Colab
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:  # If it's running in Google Colab, mount the drive
    from google.colab import drive
    drive.mount('/content/drive')
    HOME = "/content/drive/MyDrive/Image Restoration/image-restoration"
    if not os.path.exists(HOME): # If the repo isn't cloned yet, clone it
        os.chdir("/content/drive/MyDrive/Image Restoration/")
        !git clone https://github.com/davegabe/image-restoration.git
        os.chdir(HOME)
    else: # If it's already cloned, update it
        os.chdir(HOME)
        !git pull
else:  # If it's running in local machine, use the local path
    HOME = "./"
    os.chdir(HOME)

TRAINING_PATH = HOME + "Dataset/Training/"
EVALUATION_PATH = HOME + "Dataset/Evaluation/"
RESULTS_PATH = HOME + "Results/"
MODEL_PATH = HOME + "Model/"
WIDTH = 128
HEIGHT = 128
KEYWORD = "landscape"
QUANTITY = 100
EPOCH_SAVE = 25
BATCH_SIZE = 50

os.makedirs(TRAINING_PATH + "Original/", exist_ok=True)
os.makedirs(TRAINING_PATH + "Corrupted/", exist_ok=True)
os.makedirs(EVALUATION_PATH + "Original/", exist_ok=True)
os.makedirs(EVALUATION_PATH + "Corrupted/", exist_ok=True)
os.makedirs(RESULTS_PATH, exist_ok=True)
os.makedirs(MODEL_PATH, exist_ok=True)

## Data Download

In [ ]:
from data.downloader import download, adjustAndCrop, splitData
from data.corrupter import corrupt
import shutil

def doYouReallyWantToDownload():
    trainingFiles = len(os.listdir(TRAINING_PATH + "Original/")) + len(os.listdir(TRAINING_PATH + "Original/"))
    evaluationFiles = len(os.listdir(EVALUATION_PATH + "Original/")) + len(os.listdir(EVALUATION_PATH + "Corrupted/"))
    if trainingFiles + evaluationFiles > 0:
        if input("Oops, there seems to be some files already, do you want to overwrite them? (y/n)").capitalize() == "Y":
            # remove all training files
            shutil.rmtree(TRAINING_PATH + "Original/")
            shutil.rmtree(TRAINING_PATH + "Corrupted/")
            shutil.rmtree(EVALUATION_PATH + "Original/")
            shutil.rmtree(EVALUATION_PATH + "Corrupted/")
            os.makedirs(TRAINING_PATH + "Original/", exist_ok=True)
            os.makedirs(TRAINING_PATH + "Corrupted/", exist_ok=True)
            os.makedirs(EVALUATION_PATH + "Original/", exist_ok=True)
            os.makedirs(EVALUATION_PATH + "Corrupted/", exist_ok=True)
            return True
        else:
            return False
    else:
            return True


print("Generating dataset...")



if True:
    # ALTERNATIVE:
    # Dowload https://www.kaggle.com/datasets/arnaud58/landscape-pictures and put in TRAINING_PATH + "Original/"
    adjustAndCrop(TRAINING_PATH + "Original/", WIDTH, HEIGHT)
    splitData(TRAINING_PATH + "Original/", EVALUATION_PATH + "Original/")
    corrupt(TRAINING_PATH + "Original/", EVALUATION_PATH + "Original/", TRAINING_PATH + "Corrupted/", EVALUATION_PATH + "Corrupted/")
    print("Dataset generated.")
elif doYouReallyWantToDownload():
    download(TRAINING_PATH + "Original/", EVALUATION_PATH + "Original/", WIDTH, HEIGHT, KEYWORD, QUANTITY)
    corrupt(TRAINING_PATH + "Original/", EVALUATION_PATH + "Original/", TRAINING_PATH + "Corrupted/", EVALUATION_PATH + "Corrupted/")
    print("Dataset generated.")
else:
    print("Dataset not generated (already exists).")


## Training

In [3]:
from autoencoder.train import train

train(TRAINING_PATH, MODEL_PATH, EPOCH_SAVE, BATCH_SIZE, WIDTH, HEIGHT)

Epoch 1: 100%|██████████| 38/38 [00:10<00:00,  3.68it/s]
Epoch 2 | Loss 0.10894282907247543: 100%|██████████| 38/38 [00:09<00:00,  3.81it/s]
Epoch 3 | Loss 0.11107807606458664: 100%|██████████| 38/38 [00:10<00:00,  3.80it/s]
Epoch 4 | Loss 0.0861082449555397: 100%|██████████| 38/38 [00:09<00:00,  3.81it/s]
Epoch 5 | Loss 0.06863630563020706: 100%|██████████| 38/38 [00:10<00:00,  3.79it/s]
Epoch 6 | Loss 0.07144181430339813: 100%|██████████| 38/38 [00:10<00:00,  3.80it/s]
Epoch 7 | Loss 0.05975870415568352: 100%|██████████| 38/38 [00:10<00:00,  3.79it/s]
Epoch 8 | Loss 0.05519816279411316: 100%|██████████| 38/38 [00:10<00:00,  3.79it/s]
Epoch 9 | Loss 0.05328061431646347: 100%|██████████| 38/38 [00:09<00:00,  3.80it/s]
Epoch 10 | Loss 0.051912251859903336: 100%|██████████| 38/38 [00:09<00:00,  3.81it/s]
Epoch 11 | Loss 0.05298558622598648: 100%|██████████| 38/38 [00:10<00:00,  3.80it/s]
Epoch 12 | Loss 0.051912836730480194: 100%|██████████| 38/38 [00:09<00:00,  3.80it/s]
Epoch 13 | Loss

[SAVED MODEL] Epoch: 25 | Saved to ./Model/model_25.pth!


Epoch 26 | Loss 0.04614834859967232: 100%|██████████| 38/38 [00:10<00:00,  3.65it/s]
Epoch 27 | Loss 0.04672057181596756:  32%|███▏      | 12/38 [00:03<00:07,  3.64it/s]


UnboundLocalError: local variable 'output_images' referenced before assignment

## Testing

In [5]:
from autoencoder.test import test

TRAINING_PATH = HOME + "Dataset/Training/"
EVALUATION_PATH = HOME + "Dataset/Evaluation/"
test(MODEL_PATH, EVALUATION_PATH, RESULTS_PATH, BATCH_SIZE, WIDTH, HEIGHT)

RuntimeError: Error(s) in loading state_dict for AutoEncoder:
	Missing key(s) in state_dict: "encoder.5.weight", "encoder.5.bias", "decoder.5.weight", "decoder.5.bias". 
	Unexpected key(s) in state_dict: "encoder.6.weight", "encoder.6.bias", "decoder.4.weight", "decoder.4.bias", "decoder.6.weight", "decoder.6.bias". 
	size mismatch for decoder.3.weight: copying a param with shape torch.Size([64, 64, 3, 3]) from checkpoint, the shape in current model is torch.Size([32, 64, 3, 3]).
	size mismatch for decoder.3.bias: copying a param with shape torch.Size([64]) from checkpoint, the shape in current model is torch.Size([32]).

## Cleanup

In [ ]:
# remove RESULTS_PATH and MODEL_PATH and its contents
import shutil
shutil.rmtree(RESULTS_PATH)
shutil.rmtree(MODEL_PATH)
os.makedirs(RESULTS_PATH, exist_ok=True)
os.makedirs(MODEL_PATH, exist_ok=True)